In [ ]:
# |default_exp prefix

In [ ]:
# |export
import logging
import os
import signal
import subprocess
from configparser import ConfigParser
from pathlib import Path
from time import sleep

# Prefix
> Wine prefix manager for Trackmania 2020

In [ ]:
# |export
class Prefix:
    explorer_reg = "HKEY_CURRENT_USER\Software\Wine\Explorer"

    def __init__(
        self,
        name,
        width,
        height,
        prefix_path,
        prefix_template_path,
        credential_path,
        socket_port,
    ):
        self.name, self.width, self.height = name, width, height
        self.prefix_path = Path(prefix_path)
        self.prefix_template_path = Path(prefix_template_path)
        self.credential_path = Path(credential_path)
        self._copy_template()
        self.set_socket_port(socket_port)
        self._copy_credential()

    def _copy_template(self):
        cmd = (
            f"if [ -d {self.prefix_path} ]; then rm -rf {self.prefix_path}; fi"
            f" && cp -r {self.prefix_template_path} {self.prefix_path}"
        )
        subprocess.call(cmd, shell=True)

    def set_socket_port(self, socket_port):
        self.socket_port = socket_port
        config_path = (
            self.prefix_path / "drive_c/users/tmuser/OpenplanetNext/Settings.ini"
        )
        config = ConfigParser()
        config.optionxform = str
        config.read(config_path)
        config["AITool"] = {"socket_port": self.socket_port}
        with config_path.open("w+") as f:
            config.write(f, space_around_delimiters=False)
        config_path.chmod(0o666)

    def _copy_credential(self):
        cmd = (
            f"if [ -e {self.credential_path} ]; then cp {self.credential_path} "
            f'"{self.prefix_path}/drive_c/users/tmuser/AppData/Local/Ubisoft Game Launcher/user.dat"'
            "; fi"
        )
        subprocess.call(cmd, shell=True)

    def save_credential(self, filename):
        cmd = (
            f'cp "{self.prefix_path}/drive_c/users/tmuser/AppData/Local/Ubisoft Game Launcher/user.dat"'
            f" {self.credential_path}"
        )
        logging.info(f"Saving credentials to {self.credential_path}")
        subprocess.call(cmd, shell=True)

    def add_reg(self, key, value_name, data):
        cmd = (
            f"WINEPREFIX={self.prefix_path}"
            f' wine reg add "{key}" /v {value_name} /d {data} /f'
        )
        logging.debug(f"Calling cmd: {cmd}")
        subprocess.call(cmd, shell=True)

    def delete_reg(self, key, value_name):
        cmd = (
            f"WINEPREFIX={self.prefix_path}"
            f' wine reg delete "{key}" /v {value_name} /f'
        )
        logging.debug(f"Calling cmd: {cmd}")
        subprocess.call(cmd, shell=True)

    def run(self, gamepad_path):
        cmd = (
            f"USER=tmuser WINEPREFIX={self.prefix_path}"
            f' firejail --noprofile --nowhitelist="/dev/input" --whitelist="/dev/*" --whitelist="{gamepad_path}" --name={self.name}'
            f' wine explorer /desktop={self.name},{self.width}x{self.height}'
            f' "{self.prefix_path}/drive_c/Program Files (x86)/Ubisoft/Ubisoft Game Launcher/games/Trackmania/Trackmania.exe"'
            " > /dev/null 2>&1"
        )
        logging.info(f"Running cmd: {cmd}")
        self.proc = subprocess.Popen(
            cmd,
            shell=True,
            preexec_fn=os.setsid,
        )

    def running(self):
        return hasattr(self, "proc") and self.proc.poll() is None

    def hide_taskbar(self):
        """
        Call the run command once again to hide the wine taskbar
        """
        if self.running():
            subprocess.Popen(
                # Join the existing sandbox instead of creating a new one
                self.proc.args.replace("--name=", "--join="),
                shell=True,
                preexec_fn=os.setsid,
            )
        else:
            logging.warning("Prefix hide_taskbar was called but prefix is not running")

    def kill(self):
        if self.running():
            os.killpg(os.getpgid(self.proc.pid), signal.SIGKILL)
        else:
            logging.warning("Prefix kill was called but prefix is not running")

In [ ]:
# |hide
# |eval: false
from nbdev.doclinks import nbdev_export

nbdev_export()